**Experiments with Mistral7B-Instruct (baseline) and fine tuned model with Aventures texts**

The idea of the experiments consists in using the Classifier trained for authorship attribution (BertAA_Wilde_vs_Mistral) to test the capacity of the fine tuned model to mimetize Wilde's writting style.

Then, for each model we are going to generate fiction stories from the same starting lines and assign labels to each story and compare the results between the two models. The hypothesis is that for the fine tuned model the capacity of the model to distinguish between Wilde and Mistral will decay.

In [1]:
# import the required packages
import torch
#import simpletransformers
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, platform, gradio, warnings
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from huggingface_hub import notebook_login
import json
from tqdm import tqdm
import utils
import re

2024-03-04 10:46:25.263291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 10:46:25.263323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 10:46:25.264217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 10:46:25.268240: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 10:46:25.934465: W tensorflow/compiler/tf2

In [2]:
from utils import *

In [3]:
# data and models paths
dir_root = './AVENTURES' # comment this line to run in google colab
dir_data = f'{dir_root}/data'
models_path = f'{dir_root}/models'
list_to_generate_path = f'{dir_data}/story_prompts_for_evaluation.txt'
ft_model = f'{dir_root}/models/Mistral7B_fine_tuned_AVENTURES.pt'

In [5]:
#base model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [6]:
# load the lines to start the generation
texts_to_generation = []
with open(list_to_generate_path, 'r+', encoding='utf-8') as fd:
  texts_to_generation = fd.readlines()
texts_to_generation = [text[:-1] for text in texts_to_generation]

**Experiments with baseline model**

In [8]:
model = load_model(model_name)
tokenizer = load_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
author_preds, generated_texts = clf_exp(model, tokenizer, clf, texts_to_generation)

In [7]:
print(f"Predictions:\n Wilde: {author_preds.count(0)/len(author_preds) * 100} % \n Mistral7B: {author_preds.count(1)/len(author_preds) * 100} %")

In [ ]:
# saver the results
save_generated_texts(generated_texts, author_preds, model = 'baseline', data_path = dir_data)

**Experiments with fine-tuned model**

In [7]:
model = load_model(model_name, adapt = True, from_finetuned = True, model_path = ft_model)
tokenizer = load_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
texts_to_generation = ["Enfants, à vos pièces, dit Kernok en se précipitant sur son banc de quart le porte-voix à la main ; à vos pièces, et, sacredieu ! ne faites pas feu avant le commandement. L'abordage !... l'abordage !… On se suspend au cordage, On s'élance des haubans,",
                       "Que mon frère écoute son ami, la langue du chef ne sera pas fourchue, il ne dira que ce qui est vrai. — Je vous écoute, chef. Je sais que vos paroles seront celles d’un sachem parlant avec un ami. — Ce matin, un peu avant le lever du soleil, je traversais avec mes guerriers la Vallée des ombres, mes guerriers suivaient l’orée de la forêt,",
                       "Il brandille à Montfaucon, au bout d’une chaîne de fer, en attendant que sa carcasse déchiquetée des oiseaux tombe en la fosse du gibet, sur les ossements des camarades qui l’ont précédé. — C’est donc cela, dit Lampourde avec le plus beau sang-froid du monde, qu’on ne le voyait pas depuis quelque temps."]

In [8]:
def clf_exp(model, tokenizer, texts):
    """
    Function for the experiments: Given a model, a tokenizer and a classifier, it generates text using the 
    lines in the texts list and then predicts the label of the generated text using the classifier
    inputs:
      model: model: model to use
      tokenizer: tokenizer: tokenizer to use
      texts: list: list of strings with the lines to generate and predict
    outputs:
        label_predictions: list: list of predictions of the classifier
        generated_texts: list: list of the generated texts
    """

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # pattern to remove the prompt tokens after generation
    patt = r'\[INST]|\[\/INST]|\<s>|\</s>|This are the first lines of a work of fiction. Continue it.'

    generated_texts = [] # list of generated texts
    label_predictions = [] # list of predictions of the classifier
    for input in tqdm(texts):
      tokens = tokenize(tokenizer, input)
      model_inputs = tokens.to(device)
      generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True)
      decoded = tokenizer.batch_decode(generated_ids)
      decoded = [re.sub(patt, '', x) for x in decoded] # clean the generated texts
      print(decoded)
      generated_texts.extend(decoded)
      del model_inputs
      del decoded
      del generated_ids
    return label_predictions, generated_texts

In [12]:
author_preds_ft, generated_texts_ft = clf_exp(model, tokenizer, texts_to_generation)

 33%|██████████████████████████████████████████████████████████▋                                                                                                                     | 1/3 [00:20<00:40, 20.31s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["  Enfants, à vos pièces, dit Kernok en se précipitant sur son banc de quart le porte-voix à la main ; à vos pièces, et, sacredieu ! ne faites pas feu avant le commandement. L'abordage !... l'abordage !… On se suspend au cordage, On s'élance des haubans,  à chaque commandement, et il s'élance le dernier pour la rallier… et, à quelle heure, s'amuse-t-on les enfants ? À vos pièces, enfants… On fait feu… on fait feu… on fait feu… ;  on fait feu… On fait feu… on fait feu… on fait feu… on fait feu… on fait feu…  À chaque commandement, la petite Catherine se précipitait des haubans, et sans faire feu que le commandement pour l'abordage lui eût fait dire à Kernok. Le capitaine était devenue bien mécontente. Son attente n'a pas été du tout exagérée, mais si ses enfants pouvaient s'amuser au sort du capitaine ennemi, il ne fallait pas dire d'autres choses que le capitaine Kernok ? Si on pouvaient se contenter de l'amusement des siens, il ne fallait pas dire aux autres de la petite Cathy qu'un 

 67%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                          | 2/3 [00:40<00:20, 20.26s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['  Que mon frère écoute son ami, la langue du chef ne sera pas fourchue, il ne dira que ce qui est vrai. — Je vous écoute, chef. Je sais que vos paroles seront celles d’un sachem parlant avec un ami. — Ce matin, un peu avant le lever du soleil, je traversais avec mes guerriers la Vallée des ombres, mes guerriers suivaient l’orée de la forêt,  mais mon cœur écoutait l’ombre qui le projettait là-dedans. Mon coeur me donna des sensations de la plus grande terreur ; j’allai  voir ce que mon cœur voulait me montrer. Céadéux, le chef d’un des autres bandes de la tribu, m’attachait des yeux, comme un animal à sa maîtrise. À mesure qu’au fond de l’ombre il revenait à sa forme humaine, il vint faire un mouvement d’épouvante en voyant qu’au pied du peu de bûcher qui servait de foyer à notre village, restaient les corps de deux Hommes-noirs se tordus en spirale. Aucune flamme n’était allumée ici, à cause de la grande nuée d’orages qui avait passé ici la nuit. C’était de ces grands orages qui dét

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:00<00:00, 20.20s/it]

['  Il brandille à Montfaucon, au bout d’une chaîne de fer, en attendant que sa carcasse déchiquetée des oiseaux tombe en la fosse du gibet, sur les ossements des camarades qui l’ont précédé. — C’est donc cela, dit Lampourde avec le plus beau sang-froid du monde, qu’on ne le voyait pas depuis quelque temps.  Ah ! lui disait-il alors, dans la perspective du gibet que le chirurgien semblait tenir de sa main et que son adjudant, dans la même perspective de la fosse la plus large à laquelle  on avait envoyé la tête, semblait avoir entendue faire plaisir le jour de la fête, le gibet que son adjudiant le poussait auprès de lui en l’espace d’une minute, ne s’était pas fait éternellement, et au-devant des yeux même de ceux qui, comme lui, n’auront jamais prit le nom de soldat.  —Le soldat, cést’à toi qui, dans l’état dans lequel tu me rencontres, ne m’apparaisse pas pour assez d’honneur, ça vaut que je n’aurois plus eu peur de tes lames et de ta voix, mon cher Lampourde ; que j’aurais eu peur 

In [30]:
def save_generated_texts_and_labels(texts, labels, model = 'baseline', data_path = 'data'):
    """
    Save the generated texts and labels in a json file
    inputs:
      texts: list: list of generated texts
      labels: list: list of labels of the generated texts
      model: str: name of the model used to generate the texts
    """
    dict_text_to_author = {'text': [], 'label': []}

    for i in range(len(texts)):
      dict_text_to_author['text'].append(texts[i])
      #dict_text_to_author['label'].append(str(labels[i]))

    with open(f"{data_path}/{model}_generated_texts.json", 'w+') as fd:
      json.dump(dict_text_to_author, fd)

In [31]:
# saver the results
save_generated_texts_and_labels(generated_texts_ft, author_preds_ft, model = 'Mistral7B_fine_tuned', data_path = dir_data)

In [20]:
len(generated_texts_ft[0])

2185